In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import cmath
import sdeint

ヒルベルト変換によってデータから位相を取り出し、位相モデルのパラメータを最尤推定。

次のモデルを仮定してパラメータ$\omega_j, b_{jk}, \sigma_i^2$を推定。
$$\dot{\phi}_j = \omega_j + \sum_k b_{jk}\sin(\phi_k-\phi_j) + \eta_j (t)$$
を仮定する。
ただし、$\eta_j$はガウシアンホワイトノイズ。$\langle \eta_j \rangle = 0, \langle \eta_i(t_1) \eta_j(t_2) \rangle = \delta_{ij}\sigma_i^2\delta(t_1-t_2)$

データ　x[j][t] ：j番目のデータの時刻tの値。

## データxからモデルパラメータを最尤推定
注意：ノイズ強度の推定は未実装。

In [90]:
h=0.01
def MLE_phase_model(x):
    K = len(x) #振動子数
    T = len(x.T) #データ長
    analytic_signal = signal.hilbert(x)
    phase = np.angle(analytic_signal) #-piからpiへジャンプがある。
    
    #(i,j)成分が振動子i,jの位相差であるような行列。高速化のために導入する。
    Diff_phase_k = np.array([phase.T[t].reshape((1,K)) - (phase.T[t]).reshape((K,1)) for t in range(1,T)]).reshape(T-1, K, K)
    
    #
    Diff_phase_t = np.angle(analytic_signal.T[1:] / analytic_signal.T[:T-1])  #位相のジャンプがないように、解析信号の偏角の差をとることで求める。
    for j in range(K):
        A1 = np.sin(np.sum(Diff_phase_k, axis=0)[j]).reshape((1, K))
        A2 = np.sum(np.array([np.sin(Diff_phase_k[t]) * np.sin(Diff_phase_k[t]) for t in range(T-1)]), axis=0)
        A = np.append(np.append(np.array([T]).reshape((1,1)), A1).reshape((1,K+1)),   np.append(A1, A2, axis=0).T, axis=0)
        b = (1/ h ) * np.append(np.sum(Diff_phase_t.T[j]).reshape((1,1)),  np.sum(np.dot(np.ones((K,1)), np.sin(Diff_phase_t.T[j]).reshape(1,T-1)).reshape((T-1,K)) * np.sin(Diff_phase_k[:,j,:].reshape((T-1, K))), axis=0).reshape((1,K)), axis=1 ).T
        
        print("estimated parameters for oscillator(", j,  ")= \n", np.linalg.solve(A,b))
        
        

## データ生成
### 最尤推定において仮定した位相モデル

In [68]:
num_node = 2 #振動子数
omega = np.array([2*np.pi, 5*np.pi]) #固有周波数
Global_Coupling = 1 
Connectivity_Matrix = np.array([[0,1], [1,0]]) # SC_Hagmann or SC_icbm

be = 0.02


def G_whitenoise(x,t):
    return np.diag(np.ones(num_node)*be)

def phase_model(y, t): #tとyの順番
    sogosayo = np.diag(np.dot(np.sin(y.reshape((1, num_node)) - y.reshape((num_node, 1))), Connectivity_Matrix.T))
    return (omega + Global_Coupling * sogosayo )


### データ生成

In [76]:
x_init= np.array([0.1, -0.1]) #初期値
T0 = 100
tspan0 = np.linspace(0.0, T0, int(T0/h)+1)
%time result0 = sdeint.itoEuler(f=phase_model, G=G_whitenoise, y0=x_init, tspan=tspan0)


CPU times: user 300 ms, sys: 5.41 ms, total: 305 ms
Wall time: 329 ms


In [91]:
MLE_phase_model(np.cos((result0).T))

estimated parameters for oscillator( 0 )= 
 [[6.40218724]
 [1.08474006]
 [0.        ]]
estimated parameters for oscillator( 1 )= 
 [[15.57315699]
 [ 0.        ]
 [-2.57738431]]


## 疑問点など
1.  相互作用の推定は（どんなときに）可能なのか。

パラメータの値が異なるモデルが同じ定常状態に至る：
例えば、２振動子系
$$ \dot{\phi}_1 = \omega_1 + b_{12}\sin (\phi_2-\phi_1)  $$
$$ \dot{\phi}_2 = \omega_2 + b_{21}\sin (\phi_1-\phi_2) $$
は、$ |(\Delta \omega) / (b_{12} + b_{21})| \le 1$（$\Delta \omega := \omega_1-\omega_2$）なら位相固定され、
定常状態
$$ \dot{\phi_1} = \omega_1 + b_{12}\sin(\Delta \phi_*), $$ 
$$ \dot{\phi_2} = -\phi_1 + \sin( \Delta \phi_*),$$
$$ \sin( \Delta \phi_*) =  |(\Delta \omega) / (b_{12} + b_{21})| $$
に至る。$\omega_j, b_{ij}$を全て正しく推定することはできない。
